<a href="https://colab.research.google.com/github/jamescyl/myrepo/blob/main/Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit
!pip install langchain
!pip install google-generativeai langchain-google-genai
!pip install wikipedia

In [ ]:
#Bring in deps
import os
import sys
sys.path.append('/content/drive/MyDrive')
from apikey import apikey

os.environ['API_KEY'] = apikey
GOOGLE_API_KEY = apikey



In [ ]:

#import streamlit as st
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.memory import ConversationBufferMemory
from langchain.utilities import WikipediaAPIWrapper


os.environ['API_KEY'] = apikey
GOOGLE_API_KEY = apikey

# App framework
#st.title('YouTube GPT Creator')
# Insert the streamlit run command
#!streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py st_view.py
#prompt = st.text_input('Plug in your prompt here')

my_prompt = 'Wonderful world'

# Prompt templates
title_template = PromptTemplate(
    input_variables = ['topic'],
    template='write me a youtube video title about {topic}'
)

script_template = PromptTemplate(
    input_variables = ['title', 'wikipedia_research'],
    template='write me a youtube video script based on this title TITLE: {title} while leveraging this wikipedia reserch:{wikipedia_research} '
)

# Memory
title_memory = ConversationBufferMemory(input_key='topic', memory_key='chat_history')
script_memory = ConversationBufferMemory(input_key='title', memory_key='chat_history')

# Llms
llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY)
title_chain = LLMChain(llm=llm, prompt=title_template, verbose=True, output_key='title', memory=title_memory)
script_chain = LLMChain(llm=llm, prompt=script_template, verbose=True, output_key='script', memory=script_memory)

wiki = WikipediaAPIWrapper()

# Show stuff to the screen if there's a prompt
if my_prompt:
    title = title_chain.run(my_prompt)
    wiki_research = wiki.run(my_prompt)
    script = script_chain.run(title=title, wikipedia_research=wiki_research)

print(my_prompt)
print(title_memory.buffer)
print(script_memory.buffer)

"""
    st.write(title)
    st.write(script)

    with st.expander('Title History'):
        st.info(title_memory.buffer)

    with st.expander('Script History'):
        st.info(script_memory.buffer)

    with st.expander('Wikipedia Research'):
        st.info(wiki_research)
"""


#llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY)
#tweet_prompt = PromptTemplate.from_template("You are a content creator. Write me a tweet about {topic}.")
#tweet_chain = LLMChain(llm=llm, prompt=tweet_prompt, verbose=True)
#if __name__=="__main__":
#    topic = "what is streamlit?"
#    resp = tweet_chain.run(topic=topic)
#    print(resp)

In [ ]:

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage
import requests
from IPython.display import Image

image_url = '/content/drive/MyDrive/SAM_0145.JPG'
content = requests.get(image_url)
Image(content)
# Llms
llm = ChatGoogleGenerativeAI(model="gemini-pro-vision", google_api_key=GOOGLE_API_KEY)
message = HumanMessage(
    content = [
        {
            "type": "text",
            "text": "What's in this image?",
        }, # You can optionally provide text paths
        {"type": "image_url","image_url": image_url},
    ]
)
llm.invoke([message])


In [10]:
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-pro-vision", google_api_key=GOOGLE_API_KEY)
# example
message = HumanMessage(
    content=[
        {
            "type": "text",
            "text": "What's in this image?",
        },  # You can optionally provide text parts
        {"type": "image_url", "image_url": "/content/drive/MyDrive/SAM_0153.JPG"},
    ]
)
llm.invoke([message])

ValueError: Unable to process the provided image source: The provided string is not a valid URL, base64, or file path.